参考：https://www.salesanalytics.co.jp/datascience/datascience160/

# 簡単な関数を作り変換器にする

## 関数の作成

In [1]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

In [2]:
def custom_func(X):
  return(X*2)

In [3]:
X = np.array([0,1,2,3,4])
custom_func(X)

array([0, 2, 4, 6, 8])

## FunctionTransformerで変換器を作る

In [5]:
cft = FunctionTransformer(custom_func)

## 変換器を使う

In [7]:
X_trans = cft.transform(X)
X_trans

array([0, 2, 4, 6, 8])

## パラメータ付き変換器

In [8]:
def custom_func_arg(X, arg):
  return(X*arg)

In [9]:
X = np.array([0,1,2,3,4])
arg = 2
custom_func_arg(X,arg)

array([0, 2, 4, 6, 8])

In [10]:
cft = FunctionTransformer(custom_func_arg,kw_args={'arg': 2})

In [11]:
X_trans = cft.transform(X)
X_trans

array([0, 2, 4, 6, 8])

In [12]:
cft.set_params(kw_args={'arg': 3})

FunctionTransformer(func=<function custom_func_arg at 0x14037ea70>,
                    kw_args={'arg': 3})

In [13]:
X_trans = cft.transform(X)
X_trans

array([ 0,  3,  6,  9, 12])

## 逆変換付き変換器

In [14]:
# カスタム関数
def custom_func_arg(X, arg):
  return(X*arg)
# 逆関数
def custom_func_arg_inv(X, arg):
  return(X/arg)

In [15]:
cft = FunctionTransformer(
  custom_func_arg,         #カスタム関数
  custom_func_arg_inv,     #逆関数
  kw_args={'arg': 3},      #カスタム関数のパラメータ
  inv_kw_args = {'arg': 3} #逆関数のパラメータ
)

In [16]:
X_trans = cft.transform(X)
X_trans

array([ 0,  3,  6,  9, 12])

## 推定器と連携しパイプラインを学習

## ライブラリのインストール

In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
# サンプルデータ
from sklearn.datasets import fetch_california_housing
# パイプライン構築のための道具
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# 今回、変換器として利用
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
# 今回、推定器として利用
import xgboost as xgb

## データセットの読み込み

In [18]:
# データセットの読み込み
california_housing = fetch_california_housing(as_frame=True)
# 特徴量（説明変数）
X = california_housing.data
# 目的変数
y = california_housing.target

In [19]:
y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

In [20]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [21]:
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
  X, 
  y, 
  test_size=0.3, 
  random_state=123
)

## 特徴量を変換対象とそれ以外に分ける

In [22]:
feature1 = X.columns.values[:-2] #後ろから2番目までを除外
feature2 = X.columns.values[-2:] #後ろから2番目まで（'Latitude', 'Longitude'）

## パイプラインの定義

In [24]:
# カスタム関数
def custom_func(X):
  return(np.log1p(X))
# カスタム変換器
cft = FunctionTransformer(custom_func)

In [25]:
# 変換器パイプラインの定義
log_trans = ColumnTransformer(
  transformers=[("cft", cft, feature1)],
  remainder = 'passthrough',
)


In [26]:
# パイプラインの定義
num_pipeline = Pipeline(
  steps=[
    ("log_trans", log_trans),
    ("regressor", xgb.XGBRegressor()),
  ]
)

## 学習とテスト

In [27]:
# パイプラインの学習
num_pipeline.fit(X_train, y_train)

Pipeline(steps=[('log_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cft',
                                                  FunctionTransformer(func=<function custom_func at 0x16c8df490>),
                                                  array(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population',
       'AveOccup'], dtype=object))])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_b...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [28]:
# 目的変数yの予測
pred_y = num_pipeline.predict(X_test)
# R2(決定係数)
r2_score(y_test, pred_y)


0.8364427822308688